In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.signal import savgol_filter

In [4]:
# Previous Model (John Tyson, 2023)

def previous_odes(t, state, scale_factor):

    E, M = state
    
    beta = (beta_0 * K ** n + beta_1 * E ** n) / (K ** n + E ** n)
    
    dE_dt = A * E**2 - B * E + C_ptth + C_iis * M
    dM_dt = (alpha * scale_factor) * M * np.log(M0 * np.exp(beta) / M)

    return [dE_dt, dM_dt]


A = 0.0225
alpha = 0.09
B = 0.12
beta_0 = 2.1
beta_1 = 1.5
C_ptth = 0.018
C_iis = 0.12
E0 = 0.496
K = 10
M0 = 0.3
n = 2

i0 = [E0, M0]


t_end = 200
t_eval = np.linspace(0, t_end, 2000)


factors = np.linspace(0, 3, 100)
pupation_times_previous = []
max_masses_previous = []

for factor in tqdm(factors):
    sol = solve_ivp(previous_odes, (0, t_end), i0,
                    args=(factor,),
                    t_eval=t_eval, max_step=0.01)
    E_val = sol.y[0]
    M_val = sol.y[1]
    t_val = sol.t
    
    if E_val[-1] > 20:
        pupation_times_previous.append(t_val[-1])
        max_masses_previous.append(M_val.max())
    else:
        pupation_times_previous.append(np.nan)
        max_masses_previous.append(np.nan)


pupation_times_previous = np.array(pupation_times_previous)

100%|██████████| 100/100 [01:01<00:00,  1.61it/s]


In [5]:
# Simple Model

def pulse(x,A,ti,tf):
    return np.where(ti <= x <= tf, A, 0)


def simple_odes(t, i, Amp, ti, tf, scale_factor, x_stop):

    x, y = i
    F = 1 if x <= x_stop else 0  # t <= 1 else 0

    dxdt = (1/2) * x**2 - x + (1/2) * y + pulse(t, Amp, ti, tf)
    dydt = (epsilon * scale_factor) * F

    return [dxdt, dydt]



epsilon = 1.027
x_stop = 1.489
t0 = 0
t1 = 9
i0 = [0, 0]
t_end = 100
t_span = (0, t_end)
t_eval = np.linspace(0, t_end, 1000)


factors = np.linspace(0, 3, 1000)
pupation_times = []
max_masses = []

for factor in tqdm(factors):
    sol = solve_ivp(simple_odes, t_span, i0,
                    args=(0, t0, t1, factor, x_stop),
                    t_eval=t_eval, max_step=0.01)
    x_val = sol.y[0]
    y_val = sol.y[1]
    t_val = sol.t
    
    if x_val[-1] > 20:
        pupation_times.append(t_val[-1]*60/4.704704704704705)
        max_masses.append(y_val[-1])
    else:
        pupation_times.append(np.nan)
        max_masses.append(np.nan)


pupation_times = np.array(pupation_times)
pupation_times_simple = savgol_filter(pupation_times[4:], window_length=101, polyorder=1)

100%|██████████| 1000/1000 [01:15<00:00, 13.28it/s]


In [ ]:
# Model Comparison

plt.plot(factors[::10], pupation_times_previous, linestyle='-', c='blue', label='Previous model')
plt.plot(factors[4:], pupation_times_simple, linestyle='-', c='green', label='Simple model')
plt.scatter(x=1, y=60, color='red', marker='o')
plt.xlabel("fold change of growth rate")
plt.ylabel("Pupation Times (hAL3E)")
plt.xlim(left=0, right=3)
plt.ylim(bottom=0, top=200)
plt.legend()
plt.show()

In [9]:
# In silico starvation experiment using the simple model

def starvation_odes(t, i, epsilon, x_stop):

    x, y = i
    F = 1 if x <= x_stop else 0   # after reaching SN (or some multiple of SN?), stop eating # 2.5
    #if x>1:
        #print(y)

    dxdt = (1/2) * x**2 - x + (1/2) * y
    dydt = epsilon * F

    return [dxdt, dydt]

optimized_epsilon = 1.0270
optimized_x_stop = 1.4893

y0 = 0
x0 = 1 - np.sqrt(1 - y0)
i0 = [x0, y0]

t_end = 10
t_span = (0, t_end) 

target_x = 30

solution0 = solve_ivp(starvation_odes, t_span, i0, args=(optimized_epsilon, optimized_x_stop), t_eval=np.linspace(0, t_end, 1000), max_step=0.01)
E0 = solution0.y[0]
if np.any(E0 >= target_x):
    target_t_0 = solution0.t[np.isfinite(E0)].max()
else:
    target_t_0 = np.inf
print(target_t_0)

4.734734734734735


In [ ]:
different_x_stops = np.linspace(0.00001, optimized_x_stop, 100)
removal_mass = []
TTPs = []

for x_stop in different_x_stops:
    
    sol = solve_ivp(starvation_odes, t_span, i0,
                    args=(optimized_epsilon, x_stop),
                    t_eval=np.linspace(0, t_end, 1000), max_step=0.01)
    y_vals, x_vals, t_vals = sol.y[1], sol.y[0], sol.t

    if np.any(x_vals >= target_x):
        t_cross = interp1d(x_vals, t_vals, kind="linear", fill_value="extrapolate")(x_stop)
        y_cross = interp1d(y_vals, t_vals, kind="linear", fill_value="extrapolate")(t_cross)
        removal_mass.append(y_cross)
        TTP = (t_vals[-1] - t_cross) * 60 / target_t_0  # fed larvae takes 60 hours to pupate, so scaling factor = 60 / fed_target_t_0 (starved laevae takes different times to pupate)
        TTPs.append(TTP)
    else:
        removal_mass.append(np.nan)
        TTPs.append(np.nan)

In [ ]:
plt.plot(removal_mass, TTPs, linestyle="-", label="starved larvae", color="red")
plt.axvline(x=1, linestyle="--", color='orange', label="SN mass")
plt.xlabel(r"Dimensionless Mass at Stop-Feeding Time $y_{stop}$")
plt.ylabel("Time to Pupariation (hours)")
plt.xlim(0.75,3)
plt.ylim(0,100)
plt.title("Remaining Time to Pupariation After Food Removal")
plt.legend()
plt.show()